In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [66]:
data = []
fetch_images = False

for i in range(1):
    print(i)
    # Define the URL to scrape
    base_url = "https://www.vivareal.com.br"
    url = base_url + f"/venda/santa-catarina/florianopolis/?pagina={i + 1}#onde=,Santa%20Catarina,Florian%C3%B3polis,,,,,city,BR%3ESanta%20Catarina%3ENULL%3EFlorianopolis,-27.594808,-48.556915,&itl_id=1000183&itl_name=vivareal_-_botao-cta_buscar_to_vivareal_resultado-pesquisa"
    
    # Set the desired user agent
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0"
    }
    
    # Send an HTTP request with the specified user agent
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the relevant table containing property data
    table = soup.find("div", attrs={"class": "results-list js-results-list"})

    rows = table.find_all("div", class_="js-card-selector")
    
    for row in rows:
        info = row.find
    
        anuncio_url = (
            base_url +
            row.find(class_="property-card__labels-container js-main-info js-listing-labels-link").get("href")
        )

        if fetch_images:
            anuncio_resp = requests.get(anuncio_url, headers=headers)
            anuncio_soup = BeautifulSoup(anuncio_resp.content, "html.parser")
        
            images_ul = anuncio_soup.find(class_="carousel__container js-carousel-scroll")
            images_data = []
            
            for image in images_ul:
                image_tag = image.find("img")
                if not isinstance(image_tag, int):
                    images_data.append({
                        "alt": image_tag.get("alt"),
                        "src": image_tag.get("src")
                    })
        
        data.append({
            "title": row.find(class_="property-card__title js-cardLink js-card-title").text,
            "imagens": images_data if fetch_images else [],
            "url": anuncio_url,
            "endereco": row.find(class_="property-card__address").text,
            "area": (
                row.find(class_="property-card__detail-item property-card__detail-area").find(class_="property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area").text
                + row.find(class_="property-card__detail-item property-card__detail-area").find(class_="property-card__detail-text js-property-card-detail-text").text
            ),
            "quartos": (
                row.find(class_="property-card__detail-item property-card__detail-room js-property-detail-rooms").find(class_="property-card__detail-value js-property-card-value").text
                + row.find(class_="property-card__detail-item property-card__detail-room js-property-detail-rooms").find(class_="property-card__detail-text js-property-card-detail-text").text
            ),
            "banheiros": (
                row.find(class_="property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom").find(class_="property-card__detail-value js-property-card-value").text
                + row.find(class_="property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom").find(class_="property-card__detail-text js-property-card-detail-text").text
            ),
            "vagas": (
                row.find(class_="property-card__detail-item property-card__detail-garage js-property-detail-garages").find(class_="property-card__detail-value js-property-card-value").text
                + row.find(class_="property-card__detail-item property-card__detail-garage js-property-detail-garages").find(class_="property-card__detail-text js-property-card-detail-text").text
            ),
            "preco": row.find(class_="property-card__price js-property-card-prices js-property-card__price-small").p.text.strip(),
            "fonte": url
        })

0


AttributeError: 'NoneType' object has no attribute 'find_all'

In [67]:
data

[]

In [60]:
df = pd.DataFrame(data)

In [62]:
df

,title,imagens,url,endereco,area,quartos,banheiros,vagas,preco,fonte
0,"Sala/Conjunto à Venda, 70m²","[{'alt': 'Foto 1 de Sala Comercial à venda, 70...",https://www.vivareal.com.br/imovel/sala-comerc...,"Rua Felipe Schmidt, 390 - Centro, Florianópoli...",70 m²,-- Quarto,1 Banheiro,-- Vaga,R$ 300.000,https://www.vivareal.com.br/venda/santa-catari...
1,Residencial Lago Di Garda,[{'alt': 'Foto 1 de Residencial Lago Di Garda ...,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Ângelo La Porta, 141 - Centro, Florianópol...",67 m²,2 Quartos,2 Banheiros,1 Vaga,R$ 980.000,https://www.vivareal.com.br/venda/santa-catari...
2,"Apartamento com 3 Quartos à Venda, 84m²",[{'alt': 'Foto 1 de Apartamento com 3 Quartos ...,https://www.vivareal.com.br/imovel/apartamento...,"Rua Doutor Homero de Miranda Gomes, 1180 - Cap...",84 m²,3 Quartos,2 Banheiros,2 Vagas,R$ 699.900,https://www.vivareal.com.br/venda/santa-catari...
3,"Apartamento com 3 Quartos à Venda, 80m²",[{'alt': 'Foto 1 de Apartamento com 3 Quartos ...,https://www.vivareal.com.br/imovel/apartamento...,"São João do Rio Vermelho, Florianópolis - SC",80 m²,3 Quartos,2 Banheiros,1 Vaga,R$ 318.090 Preço abaixo do mercado,https://www.vivareal.com.br/venda/santa-catari...
4,"Apartamento com 2 Quartos à Venda, 66m²",[{'alt': 'Foto 1 de Apartamento com 2 Quartos ...,https://www.vivareal.com.br/imovel/apartamento...,"Rodovia Baldicero Filomeno, 2757 - Ribeirão da...",66 m²,2 Quartos,2 Banheiros,1 Vaga,R$ 757.768,https://www.vivareal.com.br/venda/santa-catari...
5,"Casa com 4 Quartos à Venda, 204m²",[{'alt': 'Foto 1 de Sobrado com 4 Quartos à ve...,https://www.vivareal.com.br/imovel/sobrado-4-q...,"Estrada Dom João Becker, 3045 - Ingleses do Ri...",204 m²,4 Quartos,4 Banheiros,2 Vagas,R$ 820.000,https://www.vivareal.com.br/venda/santa-catari...
6,"Casa com 2 Quartos à Venda, 95m²",[{'alt': 'Foto 1 de Sobrado com 2 Quartos à ve...,https://www.vivareal.com.br/imovel/sobrado-2-q...,"Travessa Nildo Neponoceno Fernandes, 203 - Ing...",95 m²,2 Quartos,3 Banheiros,2 Vagas,R$ 475.000,https://www.vivareal.com.br/venda/santa-catari...
7,"Casa com 2 Quartos à Venda, 90m²",[{'alt': 'Foto 1 de Sobrado com 2 Quartos à ve...,https://www.vivareal.com.br/imovel/sobrado-2-q...,"São João do Rio Vermelho, Florianópolis - SC",90 m²,2 Quartos,3 Banheiros,1 Vaga,R$ 399.000,https://www.vivareal.com.br/venda/santa-catari...
8,D/SENSE HOME DESIGN,[{'alt': 'Foto 1 de D/SENSE HOME DESIGN em Cen...,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Felipe Schmidt, 1162 - Centro, Florianópol...",175-468 m²,3-4 Quartos,5-6 Banheiros,2-3 Vagas,A partir de R$ 4.730.000,https://www.vivareal.com.br/venda/santa-catari...
9,"Sala/Conjunto à Venda, 30m²","[{'alt': 'Foto 1 de Sala Comercial à venda, 30...",https://www.vivareal.com.br/imovel/sala-comerc...,"Avenida Rio Branco, 354 - Centro, Florianópoli...",30 m²,-- Quarto,-- Banheiro,-- Vaga,R$ 340.000,https://www.vivareal.com.br/venda/santa-catari...
